In [1]:
'''
## 共享GPU
import tensorflow as tf
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.25)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options)) 
'''

In [1]:
#使用pandas读入数据
import pandas as pd
poems_text = pd.read_table('/mnt/data/poems_clean.txt', header=None)
poems_text.columns = ["text"]

poems_text.head()

,text
0,首春:寒随穷律变 春逐鸟声开 初风飘带柳 晚雪间花梅 碧林青旧竹 绿沼翠新苔 芝田初雁去 绮...
1,初晴落景:晚霞聊自怡 初晴弥可喜 日晃百花色 风动千林翠 池鱼跃不同 园鸟声还异 寄言博通者...
2,度秋:夏律昨留灰 秋箭今移晷 峨嵋岫初出 洞庭波渐起 桂白发幽岩 菊黄开灞涘 运流方可叹 含...
3,仪鸾殿早秋:寒惊蓟门叶 秋发小山枝 松阴背日转 竹影避风移 提壶菊花岸 高兴芙蓉池 欲知凉气...
4,山阁晚秋:山亭秋色满 岩牖凉风度 疏兰尚染烟 残菊犹承露 古石衣新苔 新巢封古树 历览情无极...


In [2]:
#数据预处理
import string
import numpy as np
poems_new = []
for line in poems_text['text']:  
     title, poem = line.split(':')     #去标题
     poem = poem.replace(' ', '')      #去空格
     poem = 'bbb' + poem               #补齐
     poems_new.append(list(poem))

In [3]:
#从原始数据到矩阵
XY=[]
for poem in poems_new:
    for i in range(len(poem)-3):
        x1 = poem[i]
        x2 = poem[i+1]
        x3 = poem[i+2]
        
        y = poem[i+3]
        XY.append([x1,x2,x3,y])
print('原始诗句：')
print(poems_text['text'][3864])
print('\n')
print('训练数据：')
print(['x1','x2','x3','y'])
for i in range(132763,132773):
    print(XY[i])

原始诗句：
静夜思:床前明月光 疑是地上霜 举头望明月 低头思故乡  


训练数据：
['x1', 'x2', 'x3', 'y']
['b', 'b', 'b', '床']
['b', 'b', '床', '前']
['b', '床', '前', '明']
['床', '前', '明', '月']
['前', '明', '月', '光']
['明', '月', '光', '疑']
['月', '光', '疑', '是']
['光', '疑', '是', '地']
['疑', '是', '地', '上']
['是', '地', '上', '霜']


In [4]:
#建立从字符到数字的映射
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(poems_new)
print(tokenizer.word_index)

vocab_size = len(tokenizer.word_index) + 1

Using TensorFlow backend.


{'b': 1, '不': 2, '人': 3, '山': 4, '风': 5, '日': 6, '无': 7, '一': 8, '云': 9, '春': 10, '花': 11, '来': 12, '何': 13, '月': 14, '水': 15, '上': 16, '有': 17, '时': 18, '中': 19, '秋': 20, '天': 21, '归': 22, '年': 23, '相': 24, '夜': 25, '知': 26, '江': 27, '君': 28, '去': 29, '长': 30, '心': 31, '见': 32, '此': 33, '白': 34, '行': 35, '自': 36, '生': 37, '客': 38, '处': 39, '为': 40, '里': 41, '空': 42, '寒': 43, '在': 44, '是': 45, '雨': 46, '下': 47, '清': 48, '如': 49, '得': 50, '落': 51, '多': 52, '高': 53, '明': 54, '远': 55, '路': 56, '门': 57, '未': 58, '声': 59, '青': 60, '家': 61, '别': 62, '南': 63, '树': 64, '尽': 65, '城': 66, '今': 67, '事': 68, '应': 69, '草': 70, '还': 71, '入': 72, '前': 73, '深': 74, '千': 75, '思': 76, '流': 77, '新': 78, '向': 79, '独': 80, '出': 81, '闲': 82, '雪': 83, '色': 84, '开': 85, '看': 86, '三': 87, '道': 88, '飞': 89, '烟': 90, '更': 91, '西': 92, '回': 93, '愁': 94, '酒': 95, '谁': 96, '万': 97, '朝': 98, '东': 99, '欲': 100, '子': 101, '玉': 102, '闻': 103, '与': 104, '满': 105, '林': 106, '望': 107, '到': 108, '马': 109, '已': 110, '重': 11

In [5]:
#数字矩阵
XY_digit = np.array(tokenizer.texts_to_sequences(XY))
X_digit = XY_digit[:, :3]
Y_digit = XY_digit[:, 3]
for i in range(132763, 132773):
     print("{:<35}".format(str(XY[i])), "\t", "{:<30}".format(str(list(X_digit[i]))),"\t", Y_digit[i])

['b', 'b', 'b', '床']                	 [1, 1, 1]                      	 533
['b', 'b', '床', '前']                	 [1, 1, 533]                    	 73
['b', '床', '前', '明']                	 [1, 533, 73]                   	 54
['床', '前', '明', '月']                	 [533, 73, 54]                  	 14
['前', '明', '月', '光']                	 [73, 54, 14]                   	 141
['明', '月', '光', '疑']                	 [54, 14, 141]                  	 430
['月', '光', '疑', '是']                	 [14, 141, 430]                 	 45
['光', '疑', '是', '地']                	 [141, 430, 45]                 	 114
['疑', '是', '地', '上']                	 [430, 45, 114]                 	 16
['是', '地', '上', '霜']                	 [45, 114, 16]                  	 203


In [6]:
#词嵌入实现逻辑回归
from keras.layers import Input, Embedding
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Embedding, Flatten
hidden_size = 256

inp = Input(shape=(3,))
x = Embedding(vocab_size, hidden_size)(inp)
x = Flatten()(x)
x = Dense(vocab_size)(x)
pred = Activation('softmax')(x)

model = Model(inp, pred)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 3, 256)            1420032   
_________________________________________________________________
flatten_1 (Flatten)          (None, 768)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5547)              4265643   
_________________________________________________________________
activation_1 (Activation)    (None, 5547)              0         
Total params: 5,685,675
Trainable params: 5,685,675
Non-trainable params: 0
_________________________________________________________________


In [8]:
#模型编译与拟合
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_digit,Y_digit,test_size=0.2,random_state=0)

from keras.optimizers import Adam
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr=0.001))
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=10000)

/root/miniconda3/envs/myconda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 640404 samples, validate on 160101 samples
Epoch 1/10
640404/640404 [==============================] - 3s 4us/step - loss: 8.1018 - val_loss: 7.0622
Epoch 2/10
640404/640404 [==============================] - 2s 4us/step - loss: 6.9687 - val_loss: 6.9328
Epoch 3/10
640404/640404 [==============================] - 2s 4us/step - loss: 6.8373 - val_loss: 6.7836
Epoch 4/10
640404/640404 [==============================] - 2s 4us/step - loss: 6.6388 - val_loss: 6.6006
Epoch 5/10
640404/640404 [==============================] - 2s 4us/step - loss: 6.4263 - val_loss: 6.4424
Epoch 6/10
640404/640404 [==============================] - 2s 4us/step - loss: 6.2338 - val_loss: 6.3186
Epoch 7/10
640404/640404 [==============================] - 2s 4us/step - loss: 6.0639 - val_loss: 6.2242
Epoch 8/10
640404/640404 [==============================] - 2s 4us/step - loss: 5.9115 - val_loss: 6.1528
Epoch 9/10
640404/640404 [==============================] - 2s 4us/step - loss: 5.7729 - val_loss: 6

In [9]:
#模型预测
sample_text = ['床', '前', '明'] 
print(sample_text)
sample_index = tokenizer.texts_to_sequences(sample_text)
print(sample_index)
word_prob = model.predict(np.array(sample_index).reshape(1, 3))[0]
print(tokenizer.index_word[word_prob.argmax()], word_prob.max())

['床', '前', '明']
[[533], [73], [54]]
月 0.31469083


In [19]:
poem_incomplete = 'bbb佳****期****如****梦****'
poem_index = []
poem_text = ''
for i in range(len(poem_incomplete)):
    current_word = poem_incomplete[i]
    
    if  current_word != '*':
        #给定的词
        index = tokenizer.word_index[current_word]
    else:
        #根据前三个词预测*
        x = poem_index[-3:]
        y = model.predict(np.expand_dims(x,axis=0))[0]
        index = y.argmax()
        current_word = tokenizer.index_word[index]
        
    poem_index.append(index)
    poem_text = poem_text + current_word
    
poem_text = poem_text[3:]
print(poem_text[0:5])
print(poem_text[5:10])
print(poem_text[10:15])
print(poem_text[15:20])

佳人不可见
期君王不知
如此意何如
梦里何处是
